# GameDev AI 25.  «Генерация 3D-моделей»


## Клонирование репозитория, загрузка всех нужных зависимостей

In [ ]:
!git clone -q https://github.com/Tencent-Hunyuan/Hunyuan3D-2.git

In [ ]:
%cd Hunyuan3D-2

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
!pip install -e . -q

In [ ]:
%cd hy3dgen/texgen/custom_rasterizer
!python3 -W ignore setup.py install --quiet

In [ ]:
%cd ../../..
%cd hy3dgen/texgen/differentiable_renderer
!python3 -W ignore setup.py install --quiet

In [ ]:
!pip install open3d -q

In [ ]:
%cd /kaggle/working/Hunyuan3D-2

### Импорт всех нужных библиотек и модулей

In [ ]:
from hy3dgen.text2image import HunyuanDiTPipeline
from hy3dgen.rembg import BackgroundRemover
from hy3dgen.shapegen import Hunyuan3DDiTFlowMatchingPipeline
from hy3dgen.texgen import Hunyuan3DPaintPipeline

import os
import time

import torch
from PIL import Image
import trimesh
import numpy as np
import open3d as o3d
import gc

## Генерация 3д модели по фото

В первый запуск будет происходить загрузка всех нужных моделей, поэтому надо подождать. В последюущие разы (в рамках одной сессии) всё будет быстрее.

In [ ]:
def img_to_3d(img_path, save_dir="/kaggle/working/output", name_model="3d_model", triangles=1900, textures=True, hi_poly=False, seed=777):

    pipeline_image_3d = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
        'tencent/Hunyuan3D-2',
        subfolder='hunyuan3d-dit-v2-0',
        use_safetensors=True,
    )
    
    image = Image.open(img_path).convert("RGBA")
    if image.mode == 'RGB':
        rembg = BackgroundRemover()
        image = rembg(image)
     
    os.makedirs(save_dir, exist_ok=True)
    
    mesh = pipeline_image_3d(
        image=image,
        num_inference_steps=50,
        octree_resolution=360,
        num_chunks=10000,
        generator=torch.manual_seed(seed),
        output_type='trimesh'
        )[0]

    if hi_poly:
        mesh.export(f'{save_dir}/{name_model}_hi.obj')

    # упрощение 3д модели до нужного кол-ва треугольников (Quadric Error Metrics)
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh.faces)
    simplified_mesh = o3d_mesh.simplify_quadric_decimation(target_number_of_triangles=triangles)
    o3d.io.write_triangle_mesh(f'{save_dir}/{name_model}_low.obj', simplified_mesh)

    # Конвертация в trimesh
    vertices = np.asarray(simplified_mesh.vertices)
    faces = np.asarray(simplified_mesh.triangles)
    trimesh_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    if textures:
        # далее генерация текстур для упрощённой модели
        
        # Это небольшой костыль для генерации текстур, скорее всего так происходит из-за jupyter-среды каггла
        %cd /kaggle/working/Hunyuan3D-2/hy3dgen/texgen/custom_rasterizer
        !python -W ignore setup.py build_ext --inplace --quiet

        # явно загружаем и конвертируем в RGBA (чтобы всегда был 4-й канал)
        image = Image.open(img_path).convert("RGBA")

        pipeline_textures_3d = Hunyuan3DPaintPipeline.from_pretrained('tencent/Hunyuan3D-2')
        mesh = pipeline_textures_3d(trimesh_mesh, image=image)
    
        mesh.export(f'{save_dir}/{name_model}_text_low.glb')

        del pipeline_textures_3d

    # очистка мусора (иначе на kaggle переполняется гпу)
    del pipeline_image_3d
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    

In [ ]:
img_to_3d(img_path="example.png")